# Сборный проект 2: 

#### Предыстория
Cтартап, который продаёт продукты питания. Нужно разобраться, как ведут себя пользователи мобильного приложения.
Дизайнеры захотели поменять шрифты во всём приложении - был проведен A/A/B-эксперимент

#### Описание данных
Таблица logs_exp:
    
    EventName — название события
    DeviceIDHash — уникальный идентификатор пользователя
    EventTimestamp — время события
    ExpId — номер эксперимента: 246 и 247 — контрольные группы, 248 — экспериментальная


#### Задача
- Провести событийный анализ, изучить воронку продаж, исследовать результаты A/A/B-эксперимента

## Содержание
1. [Подготовка данных](#data1)
2. [Изучение данных](#data2)
3. [Воронка событий](#funnel)
4. [Анализ результатов эксперимента](#tests)
5. [Summary результатов](#results)


## Подготовка данных <a name="data1"></a>

In [1]:
import pandas as pd
import numpy as np
import math as mth
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from simple_colors import *
import plotly.express as px
from plotly import graph_objects as go
import seaborn as sns
from statannot import add_stat_annotation

pd.set_option('display.float_format', '{:,.3f}'.format)

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_colwidth = 100 

In [2]:
# открываем файл
try:
    data = pd.read_csv('/datasets/logs_exp.csv', sep='\t') # Yandex путь
    
except:
    data = pd.read_csv('logs_exp.csv', sep='\t') # личный путь


FileNotFoundError: [Errno 2] No such file or directory: 'logs_exp.csv'

In [ ]:
# зададим имя датафрейма
data.name = 'data'

In [ ]:
# зададим функцию для изучения данных
def prelim_analysis(df):
    print(blue(f'Таблица {df.name}', ['bold', 'underlined']))
    display(df.head())
    print(f'\nОбщая информация')
    display(df.info())
    print(f'Явные дубликаты: {df.duplicated().sum()}\n')
    print(f'Пропуски:\n{df.isna().sum()}\n')

In [ ]:
# проведем предварительный анализ
prelim_analysis(data)

- Поменяем названия столбцов
- Переведем тип данных в столбце EventTimestamp формат даты и времени 
- Уберем явные дубликаты
- Пропусков в данных нет

In [ ]:
# изменим названия столбцов
data.columns = ['event_name', 'user_id', 'event_time', 'exp_id']

In [ ]:
# изменим формат для столбца event_time
data['event_time'] = pd.to_datetime(data['event_time'], unit='s')

# добавим новый столбец только с датой
data['event_date'] = data['event_time'].dt.date

data.head()

In [ ]:
# удалим дублирующиеся строчки
data=data.drop_duplicates()

- Изучение и подготовку данных закончили, теперь можно переходить к анализу данных

## Изучение данных <a name="data2"></a>

In [ ]:
# соберем основную информацию
print(black(f'Общая информация по данным\n', ['bold', 'underlined']))
print(f'Уникальные типы событий: {data.event_name.nunique()}')
print(f'Всего событий: {data.shape[0]:,.0f}')
print(f'Всего пользователей: {data.user_id.nunique()}') 
print(f'Среднее количество событий на пользователя: {(data.shape[0] /data.user_id.nunique()):.0f}')
print(f'Период данных: {min(data.event_date)} - {max(data.event_date)}')

In [ ]:
# построим гистограмму по дате и времени

fig = px.histogram(data['event_time'], x='event_time', nbins=50, title='События по дате и времени')
fig.update_xaxes(tickangle=20)
fig.show()

- Из гистограммы видно, что мало данных до 1го августа - уберем их, чтобы не мешали анализу
- По факту у нас данные за период 2019-08-01 - 2019-08-07

In [ ]:
# отбросим данные 
date_limit=pd.datetime(2019, 8, 1)
data_new=data.query('event_time > @date_limit')
data_new.head()

In [ ]:
# проверим, сколько пользователей и событий потеряли после отсечки
delta_events=(data.shape[0]-data_new.shape[0])/data.shape[0]
delta_users=(data.user_id.nunique()-data_new.user_id.nunique())/data.user_id.nunique()

print(f'Всего событий после отсечки: {data_new.shape[0]:,.0f}; потерянных событий: {delta_events:,.1%}')
print(f'Всего пользователей: {data_new.user_id.nunique()}; потерянных пользователей: {delta_users:,.1%}') 

In [ ]:
# проверим, что у нас пользователи из всех экспериментальных групп
print(f'Экспериментальные группы после отсечки: {data_new.exp_id.unique()}')

- После отсечки мы убрали незначительную долю событий и пользователей, все экспериментальные группы на месте

## Воронка событий <a name="funnel"></a>

In [ ]:
# посмотрим какие события есть в логах и как часто они встречаются
df = data_new.groupby('event_name').agg({'event_time':'count'}).sort_values('event_time', 
                                                                            ascending=False).reset_index()
df.columns=['event', 'count_in_log']
print(black(f'Частота событий по типу', ['bold', 'underlined']))
df

In [ ]:
# посчитаем сколько пользователей совершали каждое из событий
df2 = data_new.groupby('event_name').agg({'user_id':'nunique'}).sort_values('user_id', 
                                                                            ascending=False).reset_index()
df2.columns=['event', 'count_unique_users']
df2['conv_from_all_users']=df2['count_unique_users']/data_new['user_id'].nunique()

print(black(f'Количество уникальных пользователей совершивших то или иное событие, доля от всех пользователей', 
            ['bold', 'underlined']))
df2

- Пункт tutorial выбивается из логики последовательности событий, ибо пользователь может захотеть вызвать tutorial на любом этапе пользования приложением - можем убрать это событие из воронки
- Итоговая последовательность действий:
    - Пользователь зашел на главный экран (MainScreenAppear)
    - Увидел экран с предложениями (OffersScreenAppear)
    - Увидел экран с корзиной (CartScreenAppear)
    - Увидел экран с потверждением успешной оплаты (PaymentScreenSuccessful)

In [ ]:
# построим воронку событий

df2=df2.head(4)
fig = go.Figure(
    go.Funnel(
        y=df2['event'],
        x=df2['count_unique_users'],
    )
)
fig.show() 

In [ ]:
# посчитаем конверсию от шага к шагу
df2['conv_steps']=df2['count_unique_users']/df2['count_unique_users'].shift()
print(black(f'Конверсия от шага к шагу воронки', 
            ['bold', 'underlined']))
df2

- Наибольшее количество пользователей теряются на переходе с первого шага на второй, почти 40%
- Как видно из воронки, до оплаты доходит всего 47.7% пользователей, увидевших главный экран

## Анализ результатов эксперимента <a name="tests"></a>

In [ ]:
# посчитаем количество пользователей по группам
trials=data_new.groupby('exp_id').agg({'user_id':'nunique'}).sort_values('user_id', ascending=False)
trials.columns=['count_unique_users']
trials

In [ ]:
# проверим различие между количеством пользователей в группах
trials['count_unique_users'][247]/trials['count_unique_users'][246]-1

In [ ]:
# убедимся, что каждый пользователь попал только в одну группу
df3=data_new.groupby('user_id').agg({'exp_id':'nunique'}).sort_values('exp_id', ascending=False)

df3.head()

In [ ]:
# добавим объединенную контрольную группу
combined = data_new.query('exp_id in [246,247]')
combined['exp_id']='246+247'
data_new=data_new.append(combined)

In [ ]:
# пересчитаем количество пользователей по группам
trials=data_new.groupby('exp_id').agg({'user_id':'nunique'}).sort_values('user_id', ascending=False)
trials.columns=['count_unique_users']
trials

In [ ]:
funnel_per_group = data_new.pivot_table(index='event_name', 
                              columns='exp_id', 
                              values='user_id', 
                              aggfunc='nunique'
                             ).sort_values(246, ascending=False)
funnel_per_group = funnel_per_group.head(4)
funnel_per_group

- Различие в количестве пользователей примерно 1%, все пользователи оставались только в своей группе, для обеих групп фиксировались все события
- Перейдем к проверке гипотез равенства конверсий для событий в обеих группах:
    - Мы проверяем разницу между пропорциями, наблюдаемыми на выборках из одной генеральной совокупности
    - H0 - пропорции равны (в нашем случае пропорцией выступает конверсия для события)
    - H1 - пропорции не равны
    - Проверять гипотезы будем с помощью z-теста
    - Уровень статистической значимости - 5%, однако из-за того, что мы проводим эксперимент между 4мя группами для 4х событий (16 раз в общем, множественный тест), увеличивается вероятность ложнопозитивного результата, поэтому применим поправку Бонферрони, разделив уровень значимости на количество экспериментов 

In [ ]:
# зададим функцию для проверки гипотез равенства конверсий для событий
def hypothesis(successes1,successes2, trials1, trials2):
    p1 = successes1/trials1
    p2 = successes2/trials2
    p_combined = (successes1 + successes2) / (trials1 + trials2)
    difference = p1-p2
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / trials1 + 1 / trials2))
    distr = stats.norm(0,1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    return p_value, p1, p2

In [ ]:
# проверим гипотезу равенства конверсий для событий в контрольных группах
groups=[246,247]


def p_values_cycle(groups):
    p_values=[]
    for event in funnel_per_group.index:
        p=hypothesis(funnel_per_group.loc[event, groups[0]],
                 funnel_per_group.loc[event,groups[1]], 
                 trials.loc[groups[0]].values[0], 
                 trials.loc[groups[1]].values[0])
        p_values+=[[groups[0], groups[1],event,p[1], p[2], p[0]]]
    p_values=pd.DataFrame(p_values, columns=['group1', 'group2', 'event_name','conv_group1', 'conv_group2', 'p-value'])
    alpha=0.05
    alpha = alpha / 16 # делаем поправку Бонферонни на количество экспериментов 
    # учитываем все пары групп - по 4 эксперимента для 4 пар - 16 экспериментов в общем
    p_values['rejectH0'] = p_values['p-value'] < alpha
    print(black(f'Результаты сравнения между группами {groups[0]} и {groups[1]}', ['bold', 'underlined']))
    display(p_values)

p_values_cycle(groups)  


- Для всех событий мы не можем говорить о статистической значимости разницы конверсий в контрольных группах
- А/А тест проводится для дальнейшего корректного проведением A/B-теста, с его помощью мы видим, что разбиение на группы работает корректно

In [ ]:
# проверим гипотезу равенства конверсий для событий между контрольными и экспериментальной группой
group_list=[[246,248],[247,248]]

for groups in group_list:
    p_values_cycle(groups)

- Для всех событий мы не можем говорить о статистической значимости разницы конверсий между контрольными и экспериментальной группами

In [ ]:
# проверим гипотезу равенства конверсий для событий между контрольными и экспериментальной группой
groups=['246+247',248]

p_values_cycle(groups)

- Аналогично результатам других пар, мы не можем говорить о статистической значимости разницы конверсий для всех событий между объединенной контрольной и экспериментальной группами

## Summary результатов <a name="results"></a>

- Наибольшее количество пользователей теряются на переходе с первого шага на второй (главный экран-экран предложений), почти 40%
- До оплаты доходит всего 47.7% пользователей, увидевших главный экран
- Результаты А/A/B теста не показали статистической значимости между разницей в конверсии в контрольных и экспериментальной группе - шрифты в приложении можно менять, это никак не влияет на конверсию пользователей, ни в худшую, ни в лучшую сторону